# RICE: Breaking Through the Training Bottlenecks of Reinforcement Learning with Explanation

This notebook demonstrates how to use the RICE framework to refine a pre-trained reinforcement learning agent.

## Setup

First, let's import the necessary libraries and set up the environment.

In [ ]:
import os
import numpy as np
import torch
import matplotlib.pyplot as plt
import gymnasium as gym
from stable_baselines3 import PPO

# Import RICE components
from rice.models.state_mask import ImprovedStateMask, MaskNetwork
from rice.models.rice import RICE
from rice.environments.mujoco_envs import create_env
from rice.utils.fidelity import compute_fidelity_score, find_critical_segment
from rice.utils.explanation import random_explanation, identify_critical_states

# Set random seed for reproducibility
np.random.seed(0)
torch.manual_seed(0)

# Create output directory for results
os.makedirs('./notebook_results', exist_ok=True)

## Step 1: Create Environment and Train Base Agent

Let's create a simple environment and train a base agent using PPO. For this example, we'll use the Hopper environment, but you can substitute any environment supported by RICE.

In [ ]:
# Create environment
env_name = 'CartPole-v1'  # Simple environment for demonstration
env = gym.make(env_name)

# Train a simple agent for demonstration (normally you'd train longer)
print("Training base agent...")
agent = PPO('MlpPolicy', env, verbose=1, seed=0)
agent.learn(total_timesteps=50000)  # Train for 50k steps

# Save the agent
agent.save('./notebook_results/base_agent')
print("Base agent trained and saved.")

## Step 2: Evaluate Base Agent

Now let's evaluate the performance of our base agent to establish a baseline.

In [ ]:
def evaluate_agent(agent, env, n_episodes=10):
    """Evaluate an agent's performance"""
    rewards = []
    
    for i in range(n_episodes):
        obs, _ = env.reset()
        done = False
        total_reward = 0
        
        while not done:
            action, _ = agent.predict(obs, deterministic=True)
            obs, reward, terminated, truncated, _ = env.step(action)
            total_reward += reward
            done = terminated or truncated
            
        rewards.append(total_reward)
        print(f"Episode {i+1}: reward = {total_reward}")
    
    return rewards

# Evaluate base agent
print("Evaluating base agent...")
base_rewards = evaluate_agent(agent, env)
print(f"Base agent average reward: {np.mean(base_rewards):.2f} ± {np.std(base_rewards):.2f}")

# Plot reward distribution
plt.figure(figsize=(8, 4))
plt.hist(base_rewards, bins=10, alpha=0.7)
plt.axvline(np.mean(base_rewards), color='r', linestyle='dashed', linewidth=2)
plt.title('Base Agent Reward Distribution')
plt.xlabel('Episode Reward')
plt.ylabel('Frequency')
plt.grid(True, linestyle='--', alpha=0.7)
plt.show()

## Step 3: Train Mask Network

Now we'll train a mask network to identify critical states in the agent's trajectories.

In [ ]:
# Create and train the mask network
print("Training mask network...")
state_mask = ImprovedStateMask(
    target_agent=agent,
    observation_space=env.observation_space,
    action_space=env.action_space,
    mask_bonus=0.01  # Hyperparameter α from the paper
)

mask_net = state_mask.train(env, total_timesteps=20000)  # Train for 20k steps

# Save the mask network
torch.save(mask_net.state_dict(), './notebook_results/mask_net.pt')
print("Mask network trained and saved.")

## Step 4: Visualize Critical States

Let's visualize the critical states identified by the mask network.

In [ ]:
# Collect a trajectory
obs, _ = env.reset()
states = [obs]
rewards = []
dones = []

for _ in range(200):  # Collect up to 200 steps
    action, _ = agent.predict(obs, deterministic=True)
    next_obs, reward, terminated, truncated, _ = env.step(action)
    
    states.append(next_obs)
    rewards.append(reward)
    
    if terminated or truncated:
        break
        
    obs = next_obs

# Compute importance scores
importance_scores = state_mask.explain(states)

# Find the most critical segment
start_idx, end_idx = find_critical_segment(states, importance_scores, window_fraction=0.1)

# Plot importance scores
plt.figure(figsize=(10, 6))
plt.plot(importance_scores, label='Importance Score')
plt.axvspan(start_idx, end_idx, color='red', alpha=0.3, label='Critical Segment')
plt.xlabel('Time Step')
plt.ylabel('Importance Score')
plt.title('State Importance Scores')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.7)
plt.show()

# Print critical segment details
print(f"Critical segment: steps {start_idx} to {end_idx}")
print(f"Average importance score in critical segment: {np.mean(importance_scores[start_idx:end_idx]):.4f}")

## Step 5: Compare Explanation Methods

Let's compare our explanation method with a random baseline using the fidelity score.

In [ ]:
# Define explanation methods
def random_explanation_fn(states):
    return random_explanation(states)

def mask_explanation_fn(states):
    return state_mask.explain(states)

# Compute fidelity scores
print("Computing fidelity scores...")
random_fidelity = compute_fidelity_score(
    agent=agent,
    env=env,
    explanation_method=random_explanation_fn,
    trajectory_length=200,
    window_fraction=0.1,
    n_trajectories=5
)

mask_fidelity = compute_fidelity_score(
    agent=agent,
    env=env,
    explanation_method=mask_explanation_fn,
    trajectory_length=200,
    window_fraction=0.1,
    n_trajectories=5
)

print(f"Random explanation fidelity: {random_fidelity:.4f}")
print(f"Mask network fidelity: {mask_fidelity:.4f}")

# Plot comparison
plt.figure(figsize=(8, 5))
plt.bar(['Random', 'Mask Network'], [random_fidelity, mask_fidelity], color=['blue', 'green'])
plt.ylabel('Fidelity Score')
plt.title('Explanation Method Fidelity Comparison')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

## Step 6: Refine Agent with RICE

Now let's refine the agent using RICE.

In [ ]:
# Create RICE instance
print("Creating RICE instance...")
rice = RICE(
    pretrained_agent=agent,
    mask_net=mask_net,
    env=env,
    reset_prob=0.5,  # Probability p of resetting to critical states
    exploration_coef=0.01  # Coefficient λ for the exploration bonus
)

# Refine the agent
print("Refining agent with RICE...")
refined_agent = rice.refine(total_timesteps=50000)  # Refine for 50k steps

# Save the refined agent
refined_agent.save('./notebook_results/refined_agent')
print("Agent refined and saved.")

## Step 7: Evaluate Refined Agent

Finally, let's evaluate the refined agent and compare it with the base agent.

In [ ]:
# Evaluate refined agent
print("Evaluating refined agent...")
refined_rewards = evaluate_agent(refined_agent, env)
print(f"Refined agent average reward: {np.mean(refined_rewards):.2f} ± {np.std(refined_rewards):.2f}")

# Compare base and refined agents
plt.figure(figsize=(10, 6))

# Plot reward distributions
plt.subplot(1, 2, 1)
plt.boxplot([base_rewards, refined_rewards], labels=['Base Agent', 'Refined Agent'])
plt.ylabel('Episode Reward')
plt.title('Reward Distribution Comparison')
plt.grid(True, linestyle='--', alpha=0.7)

# Plot mean rewards
plt.subplot(1, 2, 2)
plt.bar(['Base Agent', 'Refined Agent'], [np.mean(base_rewards), np.mean(refined_rewards)], 
        yerr=[np.std(base_rewards), np.std(refined_rewards)], capsize=10)
plt.ylabel('Mean Episode Reward')
plt.title('Performance Comparison')
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

# Calculate improvement
improvement = (np.mean(refined_rewards) - np.mean(base_rewards)) / abs(np.mean(base_rewards)) * 100
print(f"Improvement: {improvement:.2f}%")

## Conclusion

In this notebook, we demonstrated how to use RICE to refine a pre-trained reinforcement learning agent. RICE leverages explanations to identify critical states and uses them to construct a mixed initial state distribution for training. By incorporating exploration from these states, RICE helps the agent break through training bottlenecks and achieve better performance.

Key steps in the RICE workflow:

1. Train a base agent
2. Train a mask network to explain the agent's behavior
3. Identify critical states using the mask network
4. Refine the agent using a mixed initial state distribution and exploration

This approach is particularly effective for environments with sparse rewards or complex dynamics where standard training methods may converge to suboptimal policies.